# Lab 4: Modeling Approaches
Now that we have cleaned our data and done an initial data exploration we can start modeling! In this lab we will fit some simple linear regressions and 
By the end of this lab you will be able to:
- Prepare data for modeling
- Fit a simple linear regression model
- Compare the performance of different models.
- Describe the differences between the primary types of machine learning algorithms and when to apply them.
- 

Outline
- Data Cleaning
- Model Types
    -  
- 

In [ ]:
#Import our data analysis library
import pandas as pd
#Import a plotting library
import matplotlib.pyplot as plt
#Import a plotting library useful for styling a plot
import seaborn as sns